## nx-arangodb

<a href="https://colab.research.google.com/github/arangodb/nx-arangodb/blob/main/docs/nx_arangodb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<div style="display: flex; align-items: center; gap: 10px;">
    <img src="https://avatars.githubusercontent.com/u/388785?s=200&v=4" alt="NetworkX" height=60px>
    <img src="https://arangodb.com/wp-content/uploads/2016/05/ArangoDB_logo_avocado_@1.png" alt="ArangoDB" height=60px>
    <img src="https://rapids.ai/images/RAPIDS-logo.png" alt="RAPIDS" height=60px>
    <img src="https://insights.virti.com/content/images/2021/09/20181218-Nvidia-Inception.png" alt="NVIDIA" height=60px>
</div>

This is a [backend to NetworkX](https://networkx.org/documentation/stable/reference/backends.html) that offers [ArangoDB](https://github.com/arangodb/arangodb) as a [Persistence Layer to NetworkX Graphs](https://arangodb.com/introducing-the-arangodb-networkx-persistence-layer/).

Additional Documentation:
- [NetworkX](https://networkx.org/documentation/stable/)
- [ArangoDB](https://docs.arangodb.com/stable/)
- [nx-cugraph](https://docs.rapids.ai/api/cugraph/nightly/nx_cugraph/nx_cugraph/)

## Package Installation

In [ ]:
%%capture
!pip install nx-arangodb

##### Optional: `nx-cugraph`

❗Note that using GPU-accelerated algorithms requires **changing the runtime**❗

`Runtime` --> `Change runtime type` --> `Hardware Accelerator`

In [ ]:
!nvidia-smi
!nvcc --version

In [ ]:
%%capture
!pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com # Requires CUDA-capable GPU

## Setting up ArangoDB

In [ ]:
%%capture
!pip install adb-cloud-connector

# Source: https://github.com/arangodb/adb-cloud-connector

In [ ]:
import os
import json

from adb_cloud_connector import get_temp_credentials

con = get_temp_credentials()

os.environ["DATABASE_HOST"] = con["url"]
os.environ["DATABASE_USERNAME"] = con["username"]
os.environ["DATABASE_PASSWORD"] = con["password"]
os.environ["DATABASE_NAME"] = con["dbName"]

# Feel free to check out your temporary database!
print(json.dumps(con, indent=4))

## Starter (CPU Example)

Hello World for `nx-arangodb`

Steps breakdown:

1. Using the Environment Variables established above, instantiate an `nxadb.Graph` that is able to connect to our ArangoDB database. In order to create an ArangoDB Graph, the `name` parameter is **required**.

2. Add two nodes, which will be stored in the `node` ArangoDB Vertex Collection, with IDs `node/1` and `node/2`.

3. Add an edge, which will be stored in the `node_to_node` ArangoDB Edge Collection, with an arbitrary ID.

4. Re-instantiate the `nxadb` Graph. Given that the data is persisted in ArangoDB, we can pick up right where we left off.

5. Run an algorithm on the graph, which will pull the data from ArangoDB.

6. Iterate over the pagerank values to store the results back on each node.

7. Clear the *local* cache of the graph.

8. Fetch the node & edge data in ArangoDB

9. Experiment with different ways of fetching a node by key.

In [ ]:
# 1. Create the Graph

import networkx as nx
import nx_arangodb as nxadb

G = nxadb.Graph(name="MyGraph", default_node_type="node")

In [ ]:
# 2. Add two nodes

G.add_node(1, foo='bar')
G.add_node(2, bar='foo')

In [ ]:
# 3. Add an edge

G.add_edge(1, 2, weight=2)

In [ ]:
# 4. Re-instantiate the Graph

G = nxadb.Graph(name="MyGraph")

print(G)

In [ ]:
# 5. Run an algorithm (CPU)
nx.config.backends.arangodb.use_gpu = False # Optional

res = nx.pagerank(G)

In [ ]:
# 6. Persist the results

for k, v in res.items():
  G.nodes[k]['pagerank'] = v

In [ ]:
# 7. Clear the local cache

G.clear()

In [ ]:
# 8. Observe the persisted results

print(G[1])
print(G[2])
print(G[1][2])

In [ ]:
# 9. Experiment with different node keys

print(G.nodes[1] == G.nodes["1"] == G.nodes["node/1"])
print(G[1][2] == G["1"][2] == G["node/1"][2])

## Starter (GPU Example)

In [ ]:
# 1. Create the NetworkX Grid Graph

G_nx = nx.grid_2d_graph(500, 500)

print(G_nx)

In [ ]:
# 2. Create the ArangoDB Grid Graph

G = nxadb.Graph(incoming_graph_data=G_nx, name="Grid")

In [ ]:
# 3. Re-instantiate the Graph

G = nxadb.Graph(name="Grid")

In [ ]:
# 4. Run an algorithm (GPU)
# See *Package Installation* to install nx-cugraph ^
nx.config.backends.arangodb.use_gpu = True

res = nx.pagerank(G)

In [ ]:
# 5. Run another algorithm (GPU, cached)

res_2 = nx.hits(G)